In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash_table
import dash
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import pandas as pd
import json
import rdflib
import time
import import_ipynb
from ipynb.fs.full.MainApp import app
from ipynb.fs.full.HelperFunctions import Log_Query_Data

In [ ]:
def extract_lon_lat(ResultListdataframe):
    
        ResultListdataframe['coord'] = ResultListdataframe['coord'].str.split("\(", expand=True)[1]
        ResultListdataframe['coord'] = ResultListdataframe['coord'].str.split("\)", expand=True)
        ResultListdataframe['lon'] = ResultListdataframe['coord'].str.split(" ", expand=True)[0].astype(float)
        ResultListdataframe['lat'] = ResultListdataframe['coord'].str.split(" ", expand=True)[1].astype(float)
        
        return ResultListdataframe
        
    

In [2]:
#function to query dbpedia endpoint
def sql(value, endpoint):
    start = time.perf_counter()
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(value)
    sparql.setReturnFormat(JSON)
    processed_results = sparql.query().convert()
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)
        
    ResultListdataframe = pd.DataFrame(out, columns=cols)
    
     
    end = time.perf_counter()
    Log_Query_Data(endpoint,round(end - start,3))
    
    
    if set(['lon','lat']).issubset(ResultListdataframe.columns): 
        return ResultListdataframe
    
    elif 'coord' in ResultListdataframe.columns:
        ResultListdataframe = extract_lon_lat(ResultListdataframe)
        return ResultListdataframe
        
    else:
        return ResultListdataframe 
   


In [3]:

#---------------------------------------------------Get-data Function--------------------------------------------------------------
def get_data_function(value, endpoint,endpointType):
        if endpointType == '1':
            start = time.perf_counter()
            g = rdflib.Graph()
            g.parse("../data/" + endpoint, format=rdflib.util.guess_format("../data/" + endpoint))
    #            qres = g.query(
    #                """select * where {<http://de.dbpedia.org/resource/Karlsruhe> ?p ?o .}""")
            qres = g.query(value)
            ResultListdataframe = pd.DataFrame (qres.bindings)
            end = time.perf_counter()
            Log_Query_Data(endpoint,round(end - start,3))
            return ResultListdataframe.to_json(date_format='iso', orient='split')
        elif endpointType == '2':
            ResultListdataframe = sql(value, endpoint)    
            return ResultListdataframe.to_json(date_format='iso', orient='split')
        else:
            return None

#---------------------------------------------------Get-data Callback--------------------------------------------------------------
@app.callback([Output("alert-value", "is_open"),Output("intermediate-value", 'children')],
               Input("submit-btn", "n_clicks"),
              [State("query-text","value"), State("query-endpoint","value"),State("endpoint-query1-type", "title")])
def get_data(n_clicks, value, endpoint,endpointType):
     if n_clicks > 0:
            if(len(value)!=0 and len(endpoint)!=0):
                return False, get_data_function(value, endpoint,endpointType)
            elif(len(value)==0 or len(endpoint)==0):
                return True, None
    
     return False, None

#---------------------------------------------------Get-data Compare Callback--------------------------------------------------------------
@app.callback([Output("alert-value2", "is_open"),Output("intermediate-value2", 'children')],
               Input("compare-btn", "n_clicks"),
              [State("query-text2","value"), State("query-endpoint2","value"),State("endpoint-query2-type", "title")])
def get_data(n_clicks, value, endpoint,endpointType):
     if n_clicks > 0:
            if(len(value)!=0 and len(endpoint)!=0):
                return False, get_data_function(value, endpoint,endpointType)
            elif(len(value)==0 or len(endpoint)==0):
                return True, None
    
     return False, None
